In [2]:
import numpy as np

def bitwise_xor(a, b):
    """Scalar XOR: a ^ b = a + b - 2 * (a & b)"""
    return a + b - 2 * (a & b)

def matrix_xor(M_a, M_b):
    """
    Computes element-wise a XOR b for numpy arrays a and b using the arithmetic rearrangement:
    M_a ^ M_b = M_a + b - 2 * (M_a & M_b)
    Assumes a and b are numpy arrays of non-negative integers with compatible shapes.
    """
    return M_a + M_b - 2 * np.bitwise_and(M_a, M_b)

def x_times_matrix_xor(x, M_a, M_b):
    """
    Computes x @ (M_a XOR M_b) where XOR is element-wise, and @ is matrix multiplication.
    Assumes x is a numpy array (matrix), and a, b are numpy arrays (matrices) of non-negative integers
    with shapes such that matrix multiplication is valid (x.shape[1] == a.shape[0], and a.shape == b.shape).
    """
    xor_ab = matrix_xor(M_a, M_b)
    return np.matmul(x, xor_ab)

In [4]:
tests_scalar = [
    (1, 2, 3),
    (3, 1, 2),
    (0, 0, 0),
    (7, 7, 0),
    (12345, 67890, 80139)
]
for a, b, expected in tests_scalar:
    impl = bitwise_xor(a, b)
    builtin = np.bitwise_xor(a, b)
    print(f"a={a}, b={b}: impl={impl} == expected={expected} == builtin={builtin}")

a=1, b=2: impl=3 == expected=3 == builtin=3
a=3, b=1: impl=2 == expected=2 == builtin=2
a=0, b=0: impl=0 == expected=0 == builtin=0
a=7, b=7: impl=0 == expected=0 == builtin=0
a=12345, b=67890: impl=80139 == expected=80139 == builtin=80139


In [5]:
a_vec = np.array([1, 3])
b_vec = np.array([2, 1])
print(matrix_xor(a_vec, b_vec))  # [3 2]
print(np.bitwise_xor(a_vec, b_vec))  # [3 2]

a_zero = np.array([0, 0])
print(matrix_xor(a_zero, a_zero))  # [0 0]

a_mixed = np.array([5, 7])
b_mixed = np.array([3, 7])
print(matrix_xor(a_mixed, b_mixed))  # [6 0]

[3 2]
[3 2]
[0 0]
[6 0]
